<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tracing, Evaluation, and Analysis of an LLM Application using Phoenix</h1>

In this tutorial we will learn how to build, observe, evaluate, and analyze a LLM powered application. This is a LLM-powered Chat on Docs application that will answer questions about <a href="https://docs.arize.com/arize/">Arize</a> from their product documentation.

Key Concepts:
1. LLM Traces are  a category of telemetry data that is used to understand the execution of LLMs and the surrounding application context (such as retrieval from vector stores, usage of external tools, etc).

2. Traces are made up of a sequence of `spans`. A span represents a unit of work or operation (think a span of time).

3. LLM Evaluations help get visbility into the performance of the application

Run the next two code blocks to launch Phoenix.

## Launch Phoenix to visualize the app

In [ ]:
!pip install -qq "arize-phoenix[experimental,llama-index]" "openai>=1" gcsfs nest_asyncio

# Import Statements
import os
from getpass import getpass
from typing import List, cast

import pandas as pd
import phoenix as px
import requests
from gcsfs import GCSFileSystem
from llama_index import (
    ServiceContext,
    StorageContext,
    load_index_from_storage,
)
from llama_index.callbacks import CallbackManager
from llama_index.embeddings import OpenAIEmbedding
from llama_index.graph_stores.simple import SimpleGraphStore
from llama_index.llms import OpenAI
from phoenix import TraceDataset
from phoenix.trace import DocumentEvaluations, SpanEvaluations
from phoenix.trace.llama_index import (
    OpenInferenceTraceCallbackHandler,
)
from phoenix.trace.utils import json_lines_to_df
from tqdm import tqdm

In [ ]:
# Run to visualize the app using Phoenix Tracing & Evals

trace_jsonl_url = "https://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/context-retrieval/trace.jsonl"
hallucination_eval_url = "https://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/context-retrieval/hallucination_eval.parquet"
qa_correctness_eval_url = "https://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/context-retrieval/qa_correctness_eval.parquet"
retrieved_documents_eval_url = "https://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/context-retrieval/retrieved_documents_eval.parquet"

response = requests.get(trace_jsonl_url)

if response.status_code == 200:
    with open("trace.jsonl", "wb") as f:
        f.write(response.content)
    json_lines = []
    with open("trace.jsonl", "r") as f:
        json_lines = cast(List[str], f.readlines())
    trace_ds = TraceDataset(json_lines_to_df(json_lines))
    px.launch_app(trace=trace_ds)
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

hallucination_eval_df = pd.read_parquet(hallucination_eval_url)
qa_correctness_eval_df = pd.read_parquet(qa_correctness_eval_url)
retrieved_documents_eval_df = pd.read_parquet(retrieved_documents_eval_url)

px.log_evaluations(
    SpanEvaluations(eval_name="Hallucination", dataframe=hallucination_eval_df),
    SpanEvaluations(eval_name="QA Correctness", dataframe=qa_correctness_eval_df),
)
px.log_evaluations(
    DocumentEvaluations(eval_name="Relevance", dataframe=retrieved_documents_eval_df)
)

px.active_session().view()

## Learn how to Build the application and add Tracing

You can see the application's traces & evals above with Phoenix. If you'd like to see how the app was built using LlamaIndex & OpenAI, the sections below cover this. It will require an OpenAI key.

In [ ]:
if (
    input(
        "You can see the application's traces & evals above with Phoenix.\nIf you'd like to see how the app was built using LlamaIndex & OpenAI, type Y to continue. \nContinue [Y/n]?"
    )
    .lower()
    .startswith("n")
):
    assert False, "notebook stopped"

In [ ]:
# Uses your OpenAI API Key
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
# Pulls down the Arize Product Documentation Knowledge Base
file_system = GCSFileSystem(project="public-assets-275721")
index_path = "arize-assets/phoenix/datasets/unstructured/llm/llama-index/arize-docs/index/"
storage_context = StorageContext.from_defaults(
    fs=file_system,
    persist_dir=index_path,
    graph_store=SimpleGraphStore(),  # prevents unauthorized request to GCS
)

# Initialize the Llamaindex App and callback handler
callback_handler = OpenInferenceTraceCallbackHandler()

service_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.0),
    embed_model=OpenAIEmbedding(model="text-embedding-ada-002"),
    callback_manager=CallbackManager(handlers=[callback_handler]),
)
index = load_index_from_storage(
    storage_context,
    service_context=service_context,
)
query_engine = index.as_query_engine()

# Asking the Application questions about the Arize product
queries = [
    "How can I query for a monitor's status using GraphQL?",
    "How do I delete a model?",
    "How much does an enterprise license of Arize cost?",
    "How do I log a prediction using the python SDK?",
]

for query in tqdm(queries):
    response = query_engine.query(query)
    print(f"Query: {query}")
    print(f"Response: {response}")

# Launch Phoenix
ds = TraceDataset.from_spans(list(callback_handler.get_spans()))
px.launch_app(trace=ds)

## Learn how to Evaluate the application using Phoenix LLM Evals

We now have visibility into the inner workings of our application. Next, let's take a look at how to use LLM evals to evaluate our application.

We will be going through a few common evaluation metrics

1. Hallucination Eval: Checks if application response was an hallucination

2. Q&A Eval: Whether or not the application answers the question correctly

3. Document Relevance Eval: Grades whether or not the documents/chunks retrieved were actually relevant to answering the query

In [ ]:
# Convert traces into workable datasets

spans_df = px.Client().get_spans_dataframe()
spans_df[["name", "span_kind", "attributes.input.value", "attributes.retrieval.documents"]].head()

from phoenix.session.evaluation import get_qa_with_reference, get_retrieved_documents

retrieved_documents_df = get_retrieved_documents(px.active_session())
queries_df = get_qa_with_reference(px.active_session())

In [ ]:
# Generating the Hallucination & Q&A Eval

import nest_asyncio
from phoenix.experimental.evals import (
    HALLUCINATION_PROMPT_RAILS_MAP,
    HALLUCINATION_PROMPT_TEMPLATE,
    QA_PROMPT_RAILS_MAP,
    QA_PROMPT_TEMPLATE,
    OpenAIModel,
    llm_classify,
)

nest_asyncio.apply()  # Speeds up OpenAI API calls

# Creating Hallucination Eval which checks if the application hallucinated
hallucination_eval = llm_classify(
    dataframe=queries_df,
    model=OpenAIModel("gpt-4", temperature=0.0),
    template=HALLUCINATION_PROMPT_TEMPLATE,
    rails=list(HALLUCINATION_PROMPT_RAILS_MAP.values()),
    provide_explanation=True,  # Makes the LLM explain its reasoning
    concurrency=4,
)
hallucination_eval["score"] = (
    hallucination_eval.label[~hallucination_eval.label.isna()] == "factual"
).astype(int)

# Creating Q&A Eval which checks if the application answered the question correctly
qa_correctness_eval = llm_classify(
    dataframe=queries_df,
    model=OpenAIModel("gpt-4", temperature=0.0),
    template=QA_PROMPT_TEMPLATE,
    rails=list(QA_PROMPT_RAILS_MAP.values()),
    provide_explanation=True,  # Makes the LLM explain its reasoning
    concurrency=4,
)

qa_correctness_eval["score"] = (
    hallucination_eval.label[~qa_correctness_eval.label.isna()] == "correct"
).astype(int)

# Logs the Evaluations to Phoenix
px.log_evaluations(
    SpanEvaluations(eval_name="Hallucination", dataframe=hallucination_eval),
    SpanEvaluations(eval_name="QA Correctness", dataframe=qa_correctness_eval),
)

In [ ]:
hallucination_eval.head()

In [ ]:
qa_correctness_eval.head()

As you can see from the results, one of the queries was flagged as a hallucination.

We can use Retrieval Relevance Evals to identify if these issues are caused by the retrieval process for RAG. We are going to use an LLM to grade whether or not the chunks retrieved are relevant to the query.

In [ ]:
# Generating Retrieval Relevance Eval

from phoenix.experimental.evals import (
    RAG_RELEVANCY_PROMPT_RAILS_MAP,
    RAG_RELEVANCY_PROMPT_TEMPLATE,
    OpenAIModel,
    llm_classify,
)

retrieved_documents_eval = llm_classify(
    dataframe=retrieved_documents_df,
    model=OpenAIModel("gpt-4", temperature=0.0),
    template=RAG_RELEVANCY_PROMPT_TEMPLATE,
    rails=list(RAG_RELEVANCY_PROMPT_RAILS_MAP.values()),
    provide_explanation=True,
)

retrieved_documents_eval["score"] = (
    retrieved_documents_eval.label[~retrieved_documents_eval.label.isna()] == "relevant"
).astype(int)

px.log_evaluations(DocumentEvaluations(eval_name="Relevance", dataframe=retrieved_documents_eval))

In [ ]:
retrieved_documents_eval.head()

Looks like we are getting a lot of irrelevant chunks of text that might be polluting the prompt sent to the LLM.

If we once again visit the UI, we will now see that Phoenix has aggregated up retrieval metrics (`precision`, `ndcg`, and `hit`). We see that our hallucinations and incorrect answers directly correlate to bad retrieval!

In [ ]:
print("The Phoenix UI:", px.active_session().url)